# Azure AI -agentit Model Context Protocol (MCP) -tuen kanssa

Tämä muistikirja esittelee, kuinka käyttää Azure AI -agentteja Model Context Protocol (MCP) -työkalujen kanssa luodaksesi älykkään agentin, joka voi hyödyntää ulkoisia MCP-palvelimia parannettujen ominaisuuksien saavuttamiseksi.


## Asenna tarvittavat NuGet-paketit

Ensiksi meidän täytyy asentaa Azure AI Agents Persistent -paketti, joka tarjoaa keskeiset toiminnot Azure AI Agents -työskentelyyn.


## Avaimeton todennus - Edut

Tämä muistikirja esittelee **avaimettoman todennuksen**, joka tarjoaa useita etuja:
- ✅ **Ei hallittavia API-avaimia** - Käyttää Azureen perustuvaa identiteettitodennusta
- ✅ **Parannettu turvallisuus** - Ei salaisuuksia tallennettuna koodiin tai konfiguraatioon
- ✅ **Automaattinen tunnistetietojen kierto** - Azure hallitsee tunnistetietojen elinkaaren
- ✅ **Roolipohjainen käyttöoikeus** - Käyttää Azuren RBAC:ia tarkkoihin käyttöoikeuksiin

`DefaultAzureCredential` käyttää automaattisesti parasta saatavilla olevaa tunnistetietolähdettä:
1. Hallinnoitu identiteetti (kun suoritetaan Azuren ympäristössä)
2. Azure CLI -tunnistetiedot (kehityksen aikana)
3. Visual Studio -tunnistetiedot
4. Ympäristömuuttujat (jos määritetty)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Asenna Azure Identity -paketti Azure-palveluiden todennusta varten käyttämällä DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Tuo tarvittavat nimialueet

Tuo tarvittavat nimialueet Azure AI Agentsille ja Azure Identitylle.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Määritä Azure AI Agent Client (Avaimeton todennus)

Määritä konfiguraatiomuuttujat ja luo PersistentAgentsClient käyttämällä **avaimetonta todennusta**:
- **projectEndpoint**: Azure AI Foundry -projektin päätepiste
- **modelDeploymentName**: Käyttöön otetun AI-mallin nimi (GPT-4.1 nano)
- **mcpServerUrl**: MCP-palvelimen URL-osoite (Microsoft Learn API)
- **mcpServerLabel**: Tunniste MCP-palvelimen tunnistamiseen
- **DefaultAzureCredential**: Käyttää hallittua identiteettiä, Azure CLI:tä tai muita tunnistuslähteitä (API-avaimia ei tarvita)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Luo MCP-työkalumääritelmä

Luo MCP-työkalumääritelmä, joka yhdistyy Microsoft Learn MCP -palvelimeen. Tämä mahdollistaa agentin pääsyn Microsoft Learn -sisältöön ja dokumentaatioon.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Luo AI-agentti

Luo pysyvä AI-agentti määritetyllä mallilla ja MCP-työkaluilla. Agentti on konfiguroitu seuraavasti:
- GPT-4.1 nano -malli
- Ohjeet käyttää MCP-työkaluja avuksi
- Pääsy Microsoft Learn MCP -palvelimelle


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Luo keskusteluketju ja lähetä viesti

Luo keskusteluketju ja lähetä käyttäjälle viesti, jossa kysytään Azure OpenAI:n ja OpenAI:n eroista. Tämä testaa agentin kykyä käyttää MCP-työkaluja tarkkojen tietojen antamiseen.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Määritä MCP-työkalun resurssit (Avainvapaa)

Määritä MCP-työkalun resurssit. Todella avainvapaata lähestymistapaa varten voit poistaa mukautetut otsikot, jos MCP-palvelin tukee Azure-identiteettiin perustuvaa todennusta. Alla oleva esimerkki näyttää, kuinka otsikoita lisätään tarvittaessa, mutta avainvapaissa tilanteissa nämä eivät välttämättä ole tarpeen.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Käynnistä agentin suoritus

Luo ja käynnistä suoritus käyttäjän viestin käsittelemiseksi. Agentti käyttää määritettyjä MCP-työkaluja ja -resursseja vastauksen luomiseen.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Seuraa suoritusta ja käsittele työkalujen hyväksynnät (Keyless)

Seuraa agentin suoritustilaa ja käsittele tarvittavat työkalujen hyväksynnät. Tämä silmukka:
1. Odottaa suorituksen valmistumista tai toimenpiteen tarvetta
2. Hyväksyy automaattisesti MCP-työkalukutsut tarvittaessa
3. Keyless-autentikoinnissa otsikoita ei välttämättä tarvita, jos MCP-palvelin tukee Azure-identiteettiä


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Näytä Keskustelun Tulokset

Hae ja näytä kaikki viestit keskusteluketjussa, mukaan lukien sekä käyttäjän kysymys että agentin vastaus. Viestit esitetään aikajärjestyksessä aikaleimojen ja roolimerkintöjen kanssa.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
